In [7]:
# ==============================================
# INITIAL SETUP: LOADING AND PREPARING THE DATA
# ==============================================
import pandas as pd
from datetime import datetime

# Data Loading and Preparation

## Data Loading
We have three datasets to analyze:
1. **withdrawals.csv** - Contains user withdrawal transactions
2. **gameplays.csv** - Records user gameplay activity  
3. **deposit.csv** - Tracks user deposits

Each dataset contains:
- `User Id`: Unique identifier for each user
- `Amount` (for financial transactions) or `Games Played` (for gameplay)
- `Datetime`: Timestamp of the activity

In [9]:
# Load all datasets
withdrawal = pd.read_csv('withdrawal.csv')
gameplays = pd.read_csv('gameplay.csv')
deposit = pd.read_csv('deposit.csv')

## Data Cleaning

Key cleaning steps:
1. Convert datetime strings to proper datetime objects
2. Handle any formatting issues in the dates
3. Remove rows with invalid dates that couldn't be parsed

This ensures we can properly filter and compare dates in our analysis.

In [10]:
# Clean and convert datetime for all datasets
for df in [withdrawal, gameplays, deposit]:
    df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y %H:%M', errors='coerce')
    df.dropna(subset=['Datetime'], inplace=True)  # Remove rows with invalid dates

# Question 1: Loyalty Points for Specific Time Slots

## Approach
We need to calculate points for specific time slots:
- Each day has two slots:
  - **S1**: 12am to 12pm (midnight to noon)
  - **S2**: 12pm to 12am (noon to midnight)

We'll analyze these specific slots:
1. 2nd October Slot S1
2. 16th October Slot S2  
3. 18th October Slot S1
4. 26th October Slot S2

In [12]:
# Define slots to analyze
slots_to_calculate = [
    ('02-10-2022', 'S1'),
    ('16-10-2022', 'S2'),
    ('18-10-2022', 'S1'),
    ('26-10-2022', 'S2')
]

# Dictionary to store results
slot_results = {}

for date, slot in slots_to_calculate:
    print(f"\nProcessing {date} {slot}...")
    
    # Determine time range
    if slot == 'S1':
        start_time = datetime.strptime(f'{date} 00:00:00', '%d-%m-%Y %H:%M:%S')
        end_time = datetime.strptime(f'{date} 12:00:00', '%d-%m-%Y %H:%M:%S')
    else:  # S2
        start_time = datetime.strptime(f'{date} 12:00:00', '%d-%m-%Y %H:%M:%S')
        end_time = datetime.strptime(f'{date} 23:59:59', '%d-%m-%Y %H:%M:%S')
    
    # Get all unique users
    all_users = set(deposit['User Id']).union(
               set(withdrawal['User Id'])).union(
               set(gameplays['User Id']))
    
    slot_data = []
    
    for user in all_users:
        # Filter user's activity in this slot
        user_deposits = deposit[(deposit['User Id'] == user) & 
                              (deposit['Datetime'] >= start_time) & 
                              (deposit['Datetime'] <= end_time)]
        
        user_withdrawals = withdrawal[(withdrawal['User Id'] == user) & 
                                    (withdrawal['Datetime'] >= start_time) & 
                                    (withdrawal['Datetime'] <= end_time)]
        
        user_games = gameplays[(gameplays['User Id'] == user) & 
                             (gameplays['Datetime'] >= start_time) & 
                             (gameplays['Datetime'] <= end_time)]
        
        # Calculate points components
        deposit_amt = user_deposits['Amount'].sum()
        withdrawal_amt = user_withdrawals['Amount'].sum()
        num_deposits = len(user_deposits)
        num_withdrawals = len(user_withdrawals)
        games_played = user_games['Games Played'].sum()
        
        # Apply loyalty point formula
        points = (0.01 * deposit_amt) + (0.005 * withdrawal_amt) + \
                (0.001 * max(num_deposits - num_withdrawals, 0)) + \
                (0.2 * games_played)
        
        slot_data.append({
            'User Id': user,
            'Loyalty Points': points,
            'Deposits': deposit_amt,
            'Withdrawals': withdrawal_amt,
            'Games Played': games_played
        })
    
    # Store results
    result_df = pd.DataFrame(slot_data).sort_values('Loyalty Points', ascending=False)
    slot_results[f"{date} {slot}"] = result_df
    print(f"Completed {date} {slot}. Top player: {result_df.iloc[0]['User Id']} with {result_df.iloc[0]['Loyalty Points']} points")


Processing 02-10-2022 S1...
Completed 02-10-2022 S1. Top player: 634.0 with 1478.355 points

Processing 16-10-2022 S2...
Completed 16-10-2022 S2. Top player: 634.0 with 1491.555 points

Processing 18-10-2022 S1...
Completed 18-10-2022 S1. Top player: 634.0 with 2723.1 points

Processing 26-10-2022 S2...
Completed 26-10-2022 S2. Top player: 714.0 with 2000.001 points


# Question 2: Monthly Loyalty Points & Rankings

## Approach
1. Calculate points for the entire month of October (1st-31st)
2. Rank players based on total points
3. For players with tied points, use games played as the tiebreaker
4. Assign ranks to all players

In [13]:
print("\nCalculating monthly loyalty points for October...")

# October date range
oct_start = datetime(2022, 10, 1)
oct_end = datetime(2022, 10, 31, 23, 59, 59)

# Get all unique users
all_users = set(deposit['User Id']).union(
           set(withdrawal['User Id'])).union(
           set(gameplays['User Id']))

monthly_data = []

for user in all_users:
    # Filter October activity
    user_deposits = deposit[(deposit['User Id'] == user) & 
                          (deposit['Datetime'] >= oct_start) & 
                          (deposit['Datetime'] <= oct_end)]
    
    user_withdrawals = withdrawal[(withdrawal['User Id'] == user) & 
                                (withdrawal['Datetime'] >= oct_start) & 
                                (withdrawal['Datetime'] <= oct_end)]
    
    user_games = gameplays[(gameplays['User Id'] == user) & 
                         (gameplays['Datetime'] >= oct_start) & 
                         (gameplays['Datetime'] <= oct_end)]
    
    # Calculate points
    deposit_amt = user_deposits['Amount'].sum()
    withdrawal_amt = user_withdrawals['Amount'].sum()
    num_deposits = len(user_deposits)
    num_withdrawals = len(user_withdrawals)
    games_played = user_games['Games Played'].sum()
    
    points = (0.01 * deposit_amt) + (0.005 * withdrawal_amt) + \
            (0.001 * max(num_deposits - num_withdrawals, 0)) + \
            (0.2 * games_played)
    
    monthly_data.append({
        'User Id': user,
        'Loyalty Points': points,
        'Games Played': games_played,
        'Deposits': deposit_amt,
        'Withdrawals': withdrawal_amt
    })

# Create rankings
monthly_rankings = pd.DataFrame(monthly_data).sort_values(
    ['Loyalty Points', 'Games Played'], 
    ascending=[False, False]
)
monthly_rankings['Rank'] = range(1, len(monthly_rankings)+1)

print("\nTop 10 Players for October:")
print(monthly_rankings.head(10)[['Rank', 'User Id', 'Loyalty Points', 'Games Played']])


Calculating monthly loyalty points for October...

Top 10 Players for October:
     Rank  User Id  Loyalty Points  Games Played
634     1      634       83843.325            24
99      2       99       23665.737            10
672     3      672       22757.780            10
212     4      212       22199.282             1
740     5      740       19211.824             2
566     6      566       19153.755           183
714     7      714       16764.234             6
421     8      421       15446.460          1557
369     9      369       14438.444            37
30     10       30       14053.375            13


# Question 3-5: Key Metrics Analysis

## Question 3: Average Deposit Amount
Simple mean calculation of all deposit amounts

## Question 4: Average Deposit Amount Per User
1. Calculate total deposits for each user  
2. Then take the mean of these user totals

## Question 5: Average Games Played Per User  
1. Calculate total games played for each user
2. Then take the mean of these user totals

In [15]:
# Q3: Average deposit amount
avg_deposit = deposit['Amount'].mean()
print(f"\nAverage deposit amount: {avg_deposit:.2f}")

# Q4: Average deposit per user
user_deposits = deposit.groupby('User Id')['Amount'].sum()
avg_deposit_per_user = user_deposits.mean()
print(f"\nAverage deposit amount per user: {avg_deposit_per_user:.2f}")

# Q5: Average games per user
user_games = gameplays.groupby('User Id')['Games Played'].sum()
avg_games_per_user = user_games.mean()
print(f"\nAverage games played per user: {avg_games_per_user:.1f} games")


Average deposit amount: 5492.19

Average deposit amount per user: 104669.65

Average games played per user: 355.3 games


# Part B: Bonus Allocation to Top 50 Players

## Context
We have ₹50,000 to distribute to the top 50 players. Need to determine the fairest allocation method.

## Allocation Methods Compared
1. **Proportional**: Distribute based on share of total points
2. **Tiered**: Fixed amounts for different rank tiers
   - Top 10: ₹1500 each
   - Next 20: ₹1000 each  
   - Next 20: ₹750 each
3. **Hybrid**: Weighted combination of:
   - 50% loyalty points
   - 30% games played
   - 20% deposit amount

In [16]:
print("\nCalculating bonus allocations...")

top_50 = monthly_rankings.head(50).copy()
total_pool = 50000

# Method 1: Proportional to points
top_50['Bonus_Proportional'] = (top_50['Loyalty Points'] / top_50['Loyalty Points'].sum()) * total_pool

# Method 2: Tiered approach
top_50['Bonus_Tiered'] = 0
top_50.loc[top_50['Rank'] <= 10, 'Bonus_Tiered'] = 1500
top_50.loc[(top_50['Rank'] > 10) & (top_50['Rank'] <= 30), 'Bonus_Tiered'] = 1000
top_50.loc[top_50['Rank'] > 30, 'Bonus_Tiered'] = 750

# Method 3: Hybrid approach
points_weight = 0.5
games_weight = 0.3
deposit_weight = 0.2

top_50['Bonus_Hybrid'] = (
    (points_weight * top_50['Loyalty Points'] / top_50['Loyalty Points'].sum()) +
    (games_weight * top_50['Games Played'] / top_50['Games Played'].sum()) +
    (deposit_weight * top_50['Deposits'] / top_50['Deposits'].sum())
) * total_pool

print("\nBonus Allocation Comparison for Top 10 Players:")
print(top_50.head(10)[['Rank', 'User Id', 'Loyalty Points', 
                      'Bonus_Proportional', 'Bonus_Tiered', 'Bonus_Hybrid']])


Calculating bonus allocations...

Bonus Allocation Comparison for Top 10 Players:
     Rank  User Id  Loyalty Points  Bonus_Proportional  Bonus_Tiered  \
634     1      634       83843.325         6638.861745          1500   
99      2       99       23665.737         1873.894625          1500   
672     3      672       22757.780         1802.000995          1500   
212     4      212       22199.282         1757.778142          1500   
740     5      740       19211.824         1521.225970          1500   
566     6      566       19153.755         1516.627965          1500   
714     7      714       16764.234         1327.421495          1500   
421     8      421       15446.460         1223.077835          1500   
369     9      369       14438.444         1143.261357          1500   
30     10       30       14053.375         1112.770917          1500   

     Bonus_Hybrid  
634   3463.810195  
99    1240.720460  
672   1459.588209  
212   1372.927109  
740   1207.353096  
566 

# Part C: Fairness Assessment & Improvements

## Current Formula
Loyalty Points = 
(0.01 × Deposit Amount) + 
(0.005 × Withdrawal Amount) + 
(0.001 × max(Deposits - Withdrawals, 0)) + 
(0.2 × Games Played)

## Strengths
1. Rewards both financial activity and gameplay
2. Simple and transparent calculation  
3. Small incentive for net depositors

## Weaknesses
1. Withdrawals earn points (counterintuitive)
2. Frequency component has minimal impact
3. Heavy depositors may dominate rankings
4. No reward for consistent daily play

## Suggested Improvements
1. Remove or make withdrawal points negative
2. Increase weight for frequency of activity  
3. Add bonus for distinct days played
4. Consider capping deposit-based points
5. Add multipliers for specific behaviors

## Revised Formula Suggestion

In [25]:
print("\nCalculating monthly loyalty points for October (New Formula)...")

new_monthly_data = []

for user in all_users:
    # Filter October activity (same as before)
    user_deposits = deposit[(deposit['User Id'] == user) & 
                          (deposit['Datetime'] >= oct_start) & 
                          (deposit['Datetime'] <= oct_end)]
    
    user_withdrawals = withdrawal[(withdrawal['User Id'] == user) & 
                                (withdrawal['Datetime'] >= oct_start) & 
                                (withdrawal['Datetime'] <= oct_end)]
    
    user_games = gameplays[(gameplays['User Id'] == user) & 
                         (gameplays['Datetime'] >= oct_start) & 
                         (gameplays['Datetime'] <= oct_end)]
    
    # Calculate components
    deposit_amt = user_deposits['Amount'].sum()
    withdrawal_amt = user_withdrawals['Amount'].sum()
    num_deposits = len(user_deposits)
    num_withdrawals = len(user_withdrawals)
    games_played = user_games['Games Played'].sum()
    distinct_days = len(user_games['Datetime'].dt.date.unique())
    
    # NEW FORMULA
    # points = (0.2 * deposit_amt) + \
    #         (0.001 * withdrawal_amt) + \
    #         (0.001 * max(num_deposits - num_withdrawals, 0)) + \
    #         (0.5 * games_played) + \
    #         (5 * distinct_days)
    
    points = (0.2 * deposit_amt) + \
            (0.001 * withdrawal_amt) + \
            (0.001 * max(num_deposits - num_withdrawals, 0)) + \
            (0.5 * games_played)
    
    
    new_monthly_data.append({
        'User Id': user,
        'New Loyalty Points': points,
        'Games Played': games_played,
        'Deposits': deposit_amt,
        'Withdrawals': withdrawal_amt,
        'Distinct Days': distinct_days
    })

# Create rankings for new formula
new_monthly_rankings = pd.DataFrame(new_monthly_data).sort_values(
    ['New Loyalty Points', 'Games Played'], 
    ascending=[False, False]
)
new_monthly_rankings['New Rank'] = range(1, len(new_monthly_rankings)+1)

print("\nTop 10 Players with New Formula:")
display(new_monthly_rankings.head(10)[['New Rank', 'User Id', 'New Loyalty Points', 'Games Played', 'Distinct Days']])


Calculating monthly loyalty points for October (New Formula)...

Top 10 Players with New Formula:


,New Rank,User Id,New Loyalty Points,Games Played,Distinct Days
672,1,672,431978.780,10,9
212,2,212,385586.572,1,1
566,3,566,364111.621,183,31
740,4,740,348064.372,2,2
714,5,714,335263.034,6,6
30,6,30,265958.695,13,10
222,7,222,257104.371,10,9
569,8,569,245575.023,38,22
99,9,99,235368.173,10,10
352,10,352,217192.938,313,31


In [30]:
print("\nCalculating bonus allocations...")

top_50 = new_monthly_rankings.head(50).copy()
total_pool = 50000

# Method 1: Proportional to points
top_50['Bonus_Proportional'] = (top_50['New Loyalty Points'] / top_50['New Loyalty Points'].sum()) * total_pool

# Method 2: Tiered approach
top_50['Bonus_Tiered'] = 0
top_50.loc[top_50['New Rank'] <= 10, 'Bonus_Tiered'] = 1500
top_50.loc[(top_50['New Rank'] > 10) & (top_50['New Rank'] <= 30), 'Bonus_Tiered'] = 1000
top_50.loc[top_50['New Rank'] > 30, 'Bonus_Tiered'] = 750

# Method 3: Hybrid approach
points_weight = 0.5
games_weight = 0.3
deposit_weight = 0.2

top_50['Bonus_Hybrid'] = (
    (points_weight * top_50['New Loyalty Points'] / top_50['New Loyalty Points'].sum()) +
    (games_weight * top_50['Games Played'] / top_50['Games Played'].sum()) +
    (deposit_weight * top_50['Deposits'] / top_50['Deposits'].sum())
) * total_pool

print("\nBonus Allocation Comparison for Top 10 Players:")
print(top_50.head(10)[['New Rank', 'User Id', 'New Loyalty Points', 
                      'Bonus_Proportional', 'Bonus_Tiered', 'Bonus_Hybrid']])


Calculating bonus allocations...

Bonus Allocation Comparison for Top 10 Players:
     New Rank  User Id  New Loyalty Points  Bonus_Proportional  Bonus_Tiered  \
672         1      672          431978.780         2584.918450          1500   
212         2      212          385586.572         2307.312049          1500   
566         3      566          364111.621         2178.808058          1500   
740         4      740          348064.372         2082.782902          1500   
714         5      714          335263.034         2006.180957          1500   
30          6       30          265958.695         1591.470622          1500   
222         7      222          257104.371         1538.487219          1500   
569         8      569          245575.023         1469.496737          1500   
99          9       99          235368.173         1408.419953          1500   
352        10      352          217192.938         1299.661137          1500   

     Bonus_Hybrid  
672   1818.27430